In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")

In [3]:
import pipit as pp
import dask.dataframe as dd

In [63]:
df = dd.from_dict({
    "Event Type": ["Enter", "Enter", "Enter", "Leave", "Leave", "Leave", "Enter", "Leave", "Enter", "Leave"]
}, npartitions=1)

df["_enter"] = (df["Event Type"] == "Enter").astype("int")
df["_leave"] = (df["Event Type"] == "Leave").astype("int")

df["depth"] = (df["_enter"].cumsum().shift() - df["_leave"].cumsum()).fillna(0).astype("int")
df["index"] = df.index
# df["matching_index"] = float("NaN")

# result = df.compute()

# grouped = result.groupby("depth")

# result.loc[result["Event Type"] == "Enter", "matching_index"] = grouped["index"].shift(-1)
# result.loc[result["Event Type"] == "Leave", "matching_index"] = grouped["index"].shift(1)
print(df.compute())

  Event Type  _enter  _leave  depth  index
0      Enter       1       0      0      0
1      Enter       1       0      1      1
2      Enter       1       0      2      2
3      Leave       0       1      2      3
4      Leave       0       1      1      4
5      Leave       0       1      0      5
6      Enter       1       0      0      6
7      Leave       0       1      0      7
8      Enter       1       0      0      8
9      Leave       0       1      0      9


In [75]:
data = {"Event Type": ["Enter", "Enter", "Enter", "Leave", "Leave", "Leave", "Enter", "Leave", "Enter", "Leave"]}
df = dd.from_dict(data, npartitions=1)

df["_enter"] = (df["Event Type"] == "Enter").astype("int")
df["_leave"] = (df["Event Type"] == "Leave").astype("int")

df["depth"] = (df["_enter"].cumsum().shift() - df["_leave"].cumsum()).fillna(0).astype("int")
df["index"] = df.index
df["index2"] = df.index
df["matching_index"] = float("NaN")

# Set the index to a known column
df = df.set_index("index2")


grouped = df.groupby("depth")

enter_mask = df["Event Type"] == "Enter"
leave_mask = df["Event Type"] == "Leave"

def apply_mask(df):
    enter_shifted = df["index"].shift(-1)
    leave_shifted = df["index"].shift(1)
    print(leave_shifted)
#     df["matching_index"] = df["matching_index"].where(~enter_mask, enter_shifted).where(~leave_mask, leave_shifted)
    return df

df = df.map_partitions(apply_mask, meta=df)

# Compute the result here
df.compute()

index2
0    NaN
1    0.0
2    1.0
3    2.0
4    3.0
5    4.0
6    5.0
7    6.0
8    7.0
9    8.0
Name: index, dtype: float64


,Event Type,_enter,_leave,depth,index,matching_index
index2,,,,,,
0,Enter,1,0,0,0,NaN
1,Enter,1,0,1,1,NaN
2,Enter,1,0,2,2,NaN
3,Leave,0,1,2,3,NaN
4,Leave,0,1,1,4,NaN
5,Leave,0,1,0,5,NaN
6,Enter,1,0,0,6,NaN
7,Leave,0,1,0,7,NaN
8,Enter,1,0,0,8,NaN


In [110]:
import polars as pl
import pandas as pd

n = 10000
df_pl = pl.DataFrame({'a':list(range(n)),'b':list(range(n))})
# %timeit df_pl.filter(pl.col("a")==n//2)

# df_pd = pd.DataFrame({'a':list(range(n)),'b':list(range(n))})
# df_pd = df_pd.set_index('a')
# %timeit df_pd.loc[n//2]

dict_pl = df_pl.partition_by('a',as_dict=True)
# %timeit dict_pl[n//2]
# dict_pl

In [111]:
data = {"Event Type": ["Enter", "Enter", "Enter", "Leave", "Leave", "Leave", "Enter", "Leave", "Enter", "Leave"]}
df = dd.from_dict(data, npartitions=1)

df["_enter"] = (df["Event Type"] == "Enter").astype(int)
df["_leave"] = (df["Event Type"] == "Leave").astype(int)

df["depth"] = (df["_enter"].cumsum().shift() - df["_leave"].cumsum()).fillna(0).astype("int")
df["index"] = df.index
df["matching_index"] = float("nan")

grouped = df.groupby("depth")

enter_mask = df["Event Type"] == "Enter"
leave_mask = df["Event Type"] == "Leave"

# Delaying the computation of shifted indices
enter_shifted = grouped["index"].shift(-1, meta=("matching_timestamp", "Float32"))
leave_shifted = grouped["index"].shift(1, meta=("matching_timestamp", "Float32"))

df["matching_index"] = df["matching_index"].mask(enter_mask, enter_shifted).mask(leave_mask, leave_shifted)
result = df.compute()
print(result)

ValueError: Not all divisions are known, can't align partitions. Please use `set_index` to set the index.. If you don't want the partitions to be aligned, and are calling `map_partitions` directly, pass `align_dataframes=False`.

In [113]:
import polars as pl
import pandas as pd

data = {
    "Event Type": ["Enter", "Enter", "Enter", "Leave", "Leave", "Leave", "Enter", "Leave", "Enter", "Leave"]
}

df = pl.from_pandas(pd.DataFrame(data).reset_index())

df = df.with_columns(
    (df["Event Type"] == "Enter").alias("_enter")
).with_columns(
    (df["Event Type"] == "Leave").alias("_leave")
).with_columns(
    (pl.col("_enter").cumsum().shift() - pl.col("_leave").cumsum()).cast(pl.Int16).fill_null(0).alias("depth")
).with_columns(
    pl.when(pl.col("Event Type") == "Enter")
    .then(pl.col("index").shift(-1))
    .otherwise(pl.col("index").shift(1))
    .over("depth").alias("_matching_event")
)
df

index,Event Type,_enter,_leave,depth,_matching_event
i64,str,bool,bool,i16,i64
0,"""Enter""",true,false,0,5
1,"""Enter""",true,false,1,4
2,"""Enter""",true,false,2,3
3,"""Leave""",false,true,2,2
4,"""Leave""",false,true,1,1
5,"""Leave""",false,true,0,0
6,"""Enter""",true,false,0,7
7,"""Leave""",false,true,0,6
8,"""Enter""",true,false,0,9


In [116]:
import duckdb
cursor = duckdb.connect()
print(cursor.execute('SELECT 42').fetchall())

[(42,)]


In [122]:
cursor.read_parquet("../examples/amg_16.parquet")

┌────────────────────┬────────────┬──────────────┬────────┬─────────┬──────────────────────────────────────────────────┐
│   Timestamp (ns)   │ Event Type │     Name     │ Thread │ Process │                    Attributes                    │
│       double       │  varchar   │   varchar    │ int64  │  int64  │                     varchar                      │
├────────────────────┼────────────┼──────────────┼────────┼─────────┼──────────────────────────────────────────────────┤
│                0.0 │ Instant    │ ProgramBegin │      0 │      12 │ {"program_name":"/g/g92/bhowmik1/AMG/test/amg"…  │
│ 46.774702821436044 │ Instant    │ ProgramBegin │      0 │      13 │ {"program_name":"/g/g92/bhowmik1/AMG/test/amg"…  │
│  71.59393288995312 │ Instant    │ ProgramBegin │      0 │      15 │ {"program_name":"/g/g92/bhowmik1/AMG/test/amg"…  │
│ 100.23150604593437 │ Instant    │ ProgramBegin │      0 │      14 │ {"program_name":"/g/g92/bhowmik1/AMG/test/amg"…  │
│ 24849.776813216795 │ Enter    

In [127]:
duckdb.sql("""
    SELECT * FROM amg_16.parquet WHERE "Timestamp (ns)" BETWEEN 1e3 AND 1e10
""")

┌────────────────────┬────────────┬──────────────┬────────┬─────────┬──────────────────────────────────────────────────┐
│   Timestamp (ns)   │ Event Type │     Name     │ Thread │ Process │                    Attributes                    │
│       double       │  varchar   │   varchar    │ int64  │  int64  │                     varchar                      │
├────────────────────┼────────────┼──────────────┼────────┼─────────┼──────────────────────────────────────────────────┤
│ 24849.776813216795 │ Enter      │ MPI_Init     │      0 │      15 │ {"region":"Region 36"}                           │
│ 24937.598704228472 │ Enter      │ MPI_Init     │      0 │      14 │ {"region":"Region 36"}                           │
│  25482.66717996398 │ Enter      │ MPI_Init     │      0 │      13 │ {"region":"Region 36"}                           │
│   26325.5664165217 │ Enter      │ MPI_Init     │      0 │      12 │ {"region":"Region 36"}                           │
│ 102906.73267158895 │ Instant  

In [140]:
duckdb.sql("DESCRIBE SELECT * FROM amg_16.parquet")

┌────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name   │ column_type │  null   │   key   │ default │  extra  │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ Timestamp (ns) │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ Event Type     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Name           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Thread         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Process        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Attributes     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [156]:
duckdb.sql("""
    SELECT COUNT(*) AS event_count FROM amg_16.parquet GROUP BY Process ORDER BY Process
""").df()

,event_count
0,261184
1,365214
2,374435
3,254201
4,260379
5,356653
6,368495
7,263055
8,261106
9,371997


In [159]:
duckdb.sql("SELECT DISTINCT(Process) FROM amg_16.parquet ORDER BY Process").df()

,Process
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [162]:
duckdb.sql("""
    SELECT MIN("Timestamp (ns)"), MAX("Timestamp (ns)") FROM amg_16.parquet
""")

┌───────────────────────┬───────────────────────┐
│ min("Timestamp (ns)") │ max("Timestamp (ns)") │
│        double         │        double         │
├───────────────────────┼───────────────────────┤
│                   0.0 │    3315728408.7821426 │
└───────────────────────┴───────────────────────┘

In [ ]:
duckdb.sql("""
    SELECT 
""")

In [103]:
df = df.with_columns(
    pl.when(pl.col("Event Type") == "Enter")
    .then(pl.col("index").shift(-1))
    .otherwise(pl.col("index").shift(1))
    .over("depth").alias("_matching_event")
)
df

index,Event Type,_enter,_leave,depth,_matching_event
i64,str,bool,bool,i64,i64
0,"""Enter""",true,false,0,5
1,"""Enter""",true,false,1,4
2,"""Enter""",true,false,2,3
3,"""Leave""",false,true,2,2
4,"""Leave""",false,true,1,1
5,"""Leave""",false,true,0,0
6,"""Enter""",true,false,0,7
7,"""Leave""",false,true,0,6
8,"""Enter""",true,false,0,9


,Event Type,_enter,_leave,depth,index,matching_index
0,Enter,1,0,0,0,5.0
1,Enter,1,0,1,1,4.0
2,Enter,1,0,2,2,3.0
3,Leave,0,1,2,3,2.0
4,Leave,0,1,1,4,1.0
5,Leave,0,1,0,5,0.0
6,Enter,1,0,0,6,7.0
7,Leave,0,1,0,7,6.0
8,Enter,1,0,0,8,9.0
9,Leave,0,1,0,9,8.0


In [115]:
df["_matching_index"] = float("NaN")
df[df["depth"] == 0].compute()
df["index"] = df.index
df["_matching_index"] = df["index"].shift()
df.compute()

,Event Type,_enter,_leave,depth,_matching_index,index
0,Enter,1,0,0,NaN,0
1,Enter,1,0,1,0.0,1
2,Enter,1,0,2,1.0,2
3,Leave,0,1,2,2.0,3
4,Leave,0,1,1,3.0,4
5,Leave,0,1,0,4.0,5
6,Enter,1,0,0,5.0,6
7,Leave,0,1,0,6.0,7
8,Enter,1,0,0,7.0,8
9,Leave,0,1,0,8.0,9


In [81]:
import dask.dataframe as dd
import pandas as pd

data = {"Event Type": ["Enter", "Enter", "Enter", "Leave", "Leave", "Leave", "Enter", "Leave", "Enter", "Leave"]}
df = dd.from_pandas(pd.DataFrame(data), npartitions=1)

df["depth"] = df["Event Type"].map({"Enter": 1}).fillna(0)

# df["depth"] = df["Event Type"].map().cumsum().astype("int")
# df.compute()
df.compute()

,Event Type,depth
0,Enter,1.0
1,Enter,1.0
2,Enter,1.0
3,Leave,0.0
4,Leave,0.0
5,Leave,0.0
6,Enter,1.0
7,Leave,0.0
8,Enter,1.0
9,Leave,0.0


In [58]:
# df["_enter"] = 0
# df["_leave"] = 0


In [59]:
df.compute()

,Event Type,_event_type
0,Enter,1
1,Enter,1
2,Enter,1
3,Leave,0
4,Leave,0
5,Leave,0
6,Enter,1
7,Leave,0
8,Enter,1
9,Leave,0


In [54]:
df

,Event Type
npartitions=1,
0,object
9,...


In [7]:
trace = pp.Trace.from_otf2("/home/rakrish/otf2-mpi-only-traces/kripke/32/")

[OTF2] src/otf2_file_posix.c:163: error: File or directory does not exist: POSIX: '/home/rakrish/otf2-mpi-only-traces/kripke/32//traces.otf2'
[OTF2] src/OTF2_Buffer.c:2047: error: File or directory does not exist: Could not open file.
[OTF2] src/OTF2_Buffer.c:2077: error: File or directory does not exist: Failed to get file handle!
[OTF2] src/OTF2_Buffer.c:586: error: File or directory does not exist: Could not read data from file to buffer!
[OTF2] src/OTF2_AnchorFile.c:164: error: File or directory does not exist: Buffer could not open file for.
[OTF2] src/OTF2_Archive.c:195: error: File or directory does not exist: Could not read data from anchor file!
[OTF2] src/OTF2_Reader.c:177: error: An error appeared when data was processed: Archive creation failed!


Error: INVALID (1): Unknown error code

In [6]:
df = trace.events
df

,Timestamp (ns),Event Type,Name,Thread,Process,Attributes
npartitions=1,,,,,,
0,float64,category[known],category[known],category[known],category[known],object
11854,...,...,...,...,...,...


In [7]:
df = df.repartition(npartitions=5)

In [8]:
df

,Timestamp (ns),Event Type,Name,Thread,Process,Attributes
npartitions=5,,,,,,
0,float64,category[known],category[known],category[known],category[known],object
2370,...,...,...,...,...,...
...,...,...,...,...,...,...
9483,...,...,...,...,...,...
11854,...,...,...,...,...,...


In [31]:
def do_sum(df):
    return df

df.groupby(["Process", "Thread"]).apply(do_sum, meta={'Timestamp (ns)': 'float'}).compute()

2023-11-13 17:59:38,238 - distributed.shuffle - ERROR - shuffle_unpack failed during shuffle 86e7f79db4164a4b22a37f3927ae7fe0
Traceback (most recent call last):
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/shuffle/_shuffle.py", line 78, in shuffle_unpack
    return _get_worker_extension().get_output_partition(
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/shuffle/_worker_extension.py", line 903, in get_output_partition
    return sync(self.worker.loop, shuffle.get_output_partition, output_partition)
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/utils.py", line 412, in sync
    raise exc.with_traceback(tb)
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/utils.py", line 385, in f
    result = yield future
  File "/home/rakrish/.local/lib/python3.8/site-packages/tornado/gen.py", line 769, in run
    value = future.result()
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/shuffle/

RuntimeError: shuffle_unpack failed during shuffle 86e7f79db4164a4b22a37f3927ae7fe0

In [97]:
df["Process"] = df["Process"].astype("int")
df = df.set_index("Process")
df = df.repartition(npartitions=64)

In [98]:
df

,Timestamp (ns),Event Type,Name,Thread,Attributes
npartitions=31,,,,,
0,float64,category[known],category[known],category[known],object
1,...,...,...,...,...
...,...,...,...,...,...
30,...,...,...,...,...
31,...,...,...,...,...


In [ ]:
import modin.pandas as pd
import numpy as np
df = pd.concat([pd.DataFrame(np.random.randint(0, 100, size=(2**20, 2**8))) for _ in range(40)]) # 40x2GB frames -- Working!
df.info()

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-11-12 14:35:03,060	INFO worker.py:1553 -- Started a local Ray instance.
